In [1]:
from vplants.autowig import autowig

In [2]:
import pystl
from path import path

rootdir = path(pystl.__path__[-1])
while not rootdir.basename() == 'src':
    rootdir = rootdir.parent
rootdir = rootdir.parent.abspath()

In [3]:
autowig.scons(rootdir, '-j3')

In [4]:
includedir = rootdir + "/build-scons/include"
headers = [str(f) for f in includedir.walkfiles('*base.h')]
    
flags = ['-x', 'c++', '-g', '-std=c++11', '-stdlib=libstdc++', '-I' + str(includedir)]

#['-x', 'c++', '-g', '-std=c++11', '-stdlib=libstdc++',
#         '-I/usr/include', '-I' + str(includedir), '-D__STDC_CONSTANT_MACROS',
#         '-I/usr/local/lib/clang/3.9.0/include', '-D__STDC_LIMIT_MACROS']

force = True

asg = autowig.AbstractSemanticGraph()
autowig.front_end.plugin = 'pyclanglite'
autowig.front_end(asg, headers, flags=flags, silent=True,
                  force_overload=True, bootstrap=0, force=force, permissive=True)
asg.clean()

In [5]:
targets = [ft for ft in asg.fundamental_types() + asg.typedefs('::pystl::*') if not ft.localname in ['void', 'null_ptr', 'unexposed', '_Complex float']]
headers = asg.headers(*targets)

In [6]:
targets

[::unsigned char,
 ::unsigned int,
 ::int,
 ::long long,
 ::unsigned short,
 ::short,
 ::unsigned long,
 ::wchar_t,
 ::unsigned long long,
 ::char32_t,
 ::signed char,
 ::char16_t,
 ::long,
 ::bool,
 ::float,
 ::char,
 ::double,
 ::long double,
 ::pystl::String,
 ::pystl::WString]

In [7]:
instantiations = {'array'         : {target: range(2,4) for target in targets},
                  'vector'        : targets,
                  'deque'         : targets,
                  'ordered_set'   : {target : ['std::less', 'std::greater'] for target in targets},
                  'ordered_map'   : {(source, target) : ['std::less', 'std::greater'] for source in targets for target in targets},
                  'unordered_set' : targets,
                  'unordered_map' : [(source, target) for source in targets for target in targets]}

In [8]:
templatedir = rootdir + '/share/template'
from mako.template import Template
templates = {template.basename().rsplit('.', 1)[0]: Template(filename=str(template)) for template in templatedir.walkfiles('*.mako')}
sourcedir = rootdir + '/src/cpp'
for template in instantiations:
    if template in templates:
        with open(sourcedir + '/' + template + '.h', 'w') as f:
            f.write(templates[template].render(headers = headers,
                                               targets = instantiations[template]))

In [9]:
autowig.scons(rootdir, '-j3')

In [10]:
asg.typedefs('::pystl::*')

[::pystl::String, ::pystl::WString]

In [11]:
headers = [str(f) for f in includedir.walkfiles('*.h')]

force = True

asg = autowig.AbstractSemanticGraph()
autowig.front_end.plugin = 'pyclanglite'
autowig.front_end(asg, headers, flags=flags, silent=True,
                  force_overload=True, bootstrap=2, cache=rootdir+'/.AutoWIG', force=force, permissive=True)

In [12]:
asg.clean()

In [13]:
autowig.back_end.plugin = 'boost_python:std_filter'
autowig.back_end(asg)

In [14]:
from vplants.autowig.boost_python_back_end import BoostPythonExportMappingFileProxy

autowig.back_end.plugin = 'boost_python:export'
autowig.back_end(asg,
                 pattern = '::pystl.*',
                 directory = rootdir + '/src/wrapper',
                 proxy = BoostPythonExportMappingFileProxy,
                 prefix = '_')
autowig.back_end.plugin = 'boost_python:closure'
autowig.back_end(asg)
autowig.back_end.plugin = 'boost_python:export'
autowig.back_end(asg,
                 directory = rootdir + '/src/wrapper',
                 proxy = BoostPythonExportMappingFileProxy,
                 prefix = '_')
autowig.back_end.plugin = 'boost_python:module'
autowig.back_end(asg,
                 filename = rootdir + '/src/wrapper/__pystl.cpp',
                 package = 'pystl')
autowig.back_end.plugin = 'boost_python:import'
autowig.back_end(asg,
                 filename = rootdir + '/src/pystl/__pystl.py',
                 module = rootdir + '/src/wrapper/__pystl.cpp')

In [ ]:
autowig.back_end.plugin = 'on_disk'
autowig.back_end(asg,
                 pattern = rootdir + '/src/(wrapper|pystl).*')

In [ ]:
autowig.scons(rootdir, '-j3')

In [ ]:
hashes = asg.nodes('struct ::std::hash')
hashes

In [ ]:
hashes[-3].header